In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari
import datetime #TODO: eliminarla se non utilizzata
import numpy as np


data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

# CREO UN NUOVO DATAFRAME PIU' VERSATILE
# Nota: Nel nostro dataframe non sono presenti valori nulli [data.isnull().sum()]

# Rimozione di colonne ridondanti e superflui
colonne_da_eliminare = ["ID", "progind", "staciv4", "posiz4", "ateco3", "ORARIO", "RAPP", "CODPROF7", "progesc", "PARTESC", "NPARTESC"]
df = data.drop(columns=colonne_da_eliminare)

# Rinomina delle colonne
df.rename(columns = dizionari.dizionario_nomi, inplace=True)

# Creazione di una colonna 'Data' nel formato (ANNO-mese)
df['Data'] = pd.to_datetime(df['Anno'].apply(str) + '-' + df['Mese'].apply(str), format='%Y-%m')
df["Data"] = df["Data"].dt.strftime("%Y-%m")

# Colonna Età nel formato (anno)
df['Età'] = df['Età'].map(dizionari.eta_mapping)
df['Regione'] = (df['Regione']/10).astype(int)

df['Zona d Italia'] = df['Zona d Italia'].map(dizionari.zona_mapping)
df['Livello di Istruzione'] = df['Livello di Istruzione'].map(dizionari.livello_di_istruzione_mapping)
df['Italia/Estero'] = df['Italia/Estero'].map(dizionari.destinazione_italia_estero_mapping)
df['Motivo dell escursione'] = df['Motivo dell escursione'].map(dizionari.motivo_escursione_mapping)
df['Sesso'] = df['Sesso'].map(dizionari.sesso_mapping)
df['Stato di impiego'] = df['Stato di impiego'].map(dizionari.stato_di_impiego_mapping)
df['Mezzo'] = df['Mezzo'].map(dizionari.mezzo_mapping)

df

,Mese,Anno,Sesso,Età,Regione,Zona d Italia,Livello di Istruzione,Stato di impiego,Impiego,Italia/Estero,Destinazione (regioni italiane o paesi esteri),Destinazione (città),Motivo dell escursione,Mezzo,Stagione,Data


In [4]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])

radio_linechart = dbc.RadioItems(
    id="radio-linecharts-id",
    options=[
        {"label": "Età", "value": "Età"},
        {"label": "Estero", "value": "Destinazione (regioni italiane o paesi esteri)"},
    ],
    value="Età",
    inline=True
)

map_anno_slider = dcc.Slider(
    2014, 2022, 1,
    value = 2018,
    id='map-anno-id',
    marks={anno: str(anno) for anno in range(2014, 2023)}
)

map_stagioni_slider = dcc.Slider(
    0, 4, 1,
    value = 2,
    id='map-stagioni-id',
    marks={
        0: 'Anno intero',
        1: 'Gennaio-Marzo',
        2: 'Aprile-Giugno',
        3: 'Luglio-Settembre',
        4: 'Ottobre-Dicembre'
        }
)


sunburst_radio =dbc.RadioItems(
    id='radio-sunburst-id',
    options=[{"label": "Zona d Italia", "value": "Zona d Italia"},
        {"label": "Livello di Istruzione", "value": "Livello di Istruzione"},
        {"label": "Sesso", "value": "Sesso"},
        ],
    value='Zona d Italia' , # Valore predefinito
    inline=True
)


color_feature_dropdown = dcc.Dropdown(
    id='color-feature',
    options=[{"label": "Zona d Italia", "value": "Zona d Italia"}],
    value='Mezzo'  # Valore predefinito
)

hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{"label": "Mezzo", "value": "Mezzo"},
        {"label": "Motivo dell escursione", "value": "Motivo dell escursione"}],
    value='Mezzo'  # Valore predefinito
)





# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi escursioni Italia"),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona la caratteristica dell'asse Y:"),
              radio_linechart,
              ])
            , width=4),
        dbc.Col(
            dbc.Stack([
              html.Label("Seleziona l'anno:"),
              map_anno_slider,
              html.Label("Seleziona la stagione:"),
              map_stagioni_slider,
            ])
        , width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=4),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='map_partenze')
            ])
        , width=4),
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='map_arrivi')
            ])
        , width=4),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                html.Label("Select a feature to visualize with a sunburst:"),
                sunburst_radio,
            ])
        ], width=4, align="center"),
        dbc.Col([
            dbc.Stack([
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              hist_feature_dropdown,
              html.Label("Seleziona le variabili per la heatmap di correlazione:"),
              color_feature_dropdown,
            ])
        ], width=8, align="center"),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='sunburst-graph')
            ])
        ], width=4),
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        ], width=8),
    ]),
])

# LINECHART - Evoluzione degli spostamenti negli anni (suddivisi per: [Età, Stati Esteri])
# Callback per il grafico a dispersione (scatter plot)
@app.callback(
    Output('graph', 'figure'),
    [Input('radio-linecharts-id', 'value')]
)
def update_line_chart(radio_linechart):
    conteggio = df.groupby(['Anno', radio_linechart]).size().reset_index(name='Numero escursioni')
    colori = ['#FF0000', '#FF4500', '#FF8C00', '#FFD700', '#ADFF2F', '#7FFF00', '#32CD32', '#008000']

    if radio_linechart == "Destinazione (regioni italiane o paesi esteri)":
        radio_linechart = "Stati Esteri"
        conteggio.rename(columns={"Destinazione (regioni italiane o paesi esteri)" : radio_linechart}, inplace=True)
        conteggio = conteggio[conteggio[radio_linechart] > 100]
        conteggio['Stati Esteri'] = conteggio[radio_linechart].map(dizionari.destinazione_regioni_stato_mapping)
        
        colori = [
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6'
            ]

    fig = px.line(conteggio,
                  x='Anno',
                  y='Numero escursioni',
                  color = radio_linechart,
                  title = f'Evoluzione degli spostamenti negli anni rispetto a [{radio_linechart}]',
                  color_discrete_sequence=colori)

    fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))
    
    return fig


@app.callback(
    Output('map_partenze', 'figure'),
    Output('map_arrivi', 'figure'),
    [Input('map-anno-id', 'value'),
     Input('map-stagioni-id', 'value')]
)
def update_map(map_anno, map_stagioni):
    maschera_anno = df["Anno"] == map_anno
    
    if map_stagioni == 0: # 0 : 'Anno intero'
        maschera_stagioni = True
    else:
        maschera_stagioni = df['Stagione'] == map_stagioni

    conteggio_partenze = df[maschera_anno & maschera_stagioni]['Regione'].value_counts().reset_index(name="N. Partenze")

    fig1 = px.choropleth(
        conteggio_partenze,
        geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson',
        locations='Regione',
        color='N. Partenze',
        color_continuous_scale='Blues',
        featureidkey='properties.reg_istat_code_num',
        #animation_frame='Date',
        range_color=(0, max(conteggio_partenze['N. Partenze']))
        )
    
    fig1.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig1.update_geos(fitbounds="locations", visible=True)

    conteggio_arrivi = df[maschera_anno & maschera_stagioni]['Destinazione (città)'].value_counts().reset_index(name="N. Arrivi")
   
    fig2 = px.choropleth(
        conteggio_arrivi,
        geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
        locations='Destinazione (città)',
        color='N. Arrivi',
        color_continuous_scale='Reds',
        featureidkey='properties.prov_istat_code_num',
        #animation_frame='Date',
        range_color=(0, max(conteggio_arrivi['N. Arrivi']))
        )
    fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig2.update_geos(fitbounds="locations", visible=True)

    
    return fig1,fig2

# Callback per il sunburst chart
@app.callback(
    Output('sunburst-graph', 'figure'),
    Input('radio-sunburst-id', 'value')
)

def update_sunburst(sunburst_radio):
    if sunburst_radio == "Zona d Italia":
        category_list = ['Zona d Italia', 'Livello di Istruzione', 'Italia/Estero']
    elif sunburst_radio == "Livello di Istruzione":
        category_list = ['Livello di Istruzione', "Zona d Italia", 'Italia/Estero']
    elif sunburst_radio == "Sesso":
        category_list = ["Sesso", 'Zona d Italia', 'Stato di impiego']

    # Generate the sunburst chart
    fig_sunburst = px.sunburst(df, path=category_list)
    
    fig_sunburst.update_layout(margin=dict(l=50, r=50, t=50, b=50))
    return fig_sunburst


@app.callback(
    Output('histogram', 'figure'),
    [Input('hist-feature', 'value'),
     Input('color-feature', 'value')]
)


def update_histogram(selected_feature, color_feature):
    labels = {selected_feature: dizionari.dizionario_nomi[selected_feature]}
    if color_feature:
        labels[color_feature] = dizionari.dizionario_nomi[color_feature]
    
    fig = px.histogram(
        df, x=selected_feature, color=color_feature,
        opacity=0.7,  # Imposta l'opacità per una migliore visibilità
        title=f'Istogramma di {dizionari.dizionario_nomi[selected_feature]}',
        labels=labels
    )
    
    fig.update_layout(
        bargap=0.01,  # Imposta lo spazio tra le barre
        bargroupgap=0.05,  # Imposta lo spazio tra i gruppi di barre
        xaxis_title=dizionari.dizionario_nomi[selected_feature],  # Imposta il titolo dell'asse x
        yaxis_title='Conteggio',  # Imposta il titolo dell'asse y
        showlegend=True,  # Mostra la legenda
        margin=dict(l=0, r=0, t=50, b=30)  # Rimuove i margini
    )
    return fig


# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external", port = "8051")


#a

Dash app running on http://127.0.0.1:8051/


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[4], line 241, in update_histogram(
    selected_feature='Mezzo',
    color_feature=None
)
    233 @app.callback(
    234     Output('histogram', 'figure'),
    235     [Input('hist-feature', 'value'),
   (...)
    239 
    240 def update_histogram(selected_feature, color_feature):
--> 241     labels = {selected_feature: dizionari.dizionario_nomi[selected_feature]}
        selected_feature = 'Mezzo'
        dizionari.dizionario_nomi = {'ID': 'ID', 'mese': 'Mese', 'annrif': 'Anno', 'progind': 'Indice progressivo', 'sesso': 'Sesso', 'eta10': 'Età', 'staciv4': 'Stato civile', 'reg': 'Regione', 'rip': 'Zona d Italia', 'istr4': 'Livello di Istruzione', 'condogg': 'Stato di impiego', 'posiz4': 'Grado di impiego', 'ateco3': 'Settore di lavoro', 'cond4': 'Impiego', 'ORARIO': 'Tipo di impiego', 'RAPP': 'Rapporto di lavoro', 'CODPROF7': 'C